<a href="https://colab.research.google.com/github/Pranavi-chowdary99/Pranavi_HDS/blob/main/midterm/midterm-2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [57]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    (string, int, float, float, float, float, float, string, string, string) -> float

    Calculates the PRIEST score using patient clinical data to evaluate the probability of negative clinical outcomes.
    This function combines patient demographic and clinical characteristics to generate a score, which is then translated into a risk likelihood.
    Adjustments for case sensitivity in string inputs are handled, and invalid sex inputs generate a ValueError.
    The scoring system follows the criteria stated on the MDCalc website for the PRIEST COVID-19 Clinical Severity Score.
    Returns a single probability value expressing the possibility of an adverse result within 30 days based on the calculated score.
    This function calculates a score based on clinical parameters and converts it to a chance of unfavorable outcome using predefined criteria.

    Args:
        sex (str): Gender assigned at birth, accepts 'male' or 'female'.
        age (int): Age of the patient in years.
        respiratory_rate (float): Respiratory rate in breaths per minute.
        oxygen_saturation (float): Oxygen saturation as a decimal between 0 and 1.
        heart_rate (float): Heart rate in beats per minute.
        systolic_bp (float): Systolic blood pressure in mmHg.
        temperature (float): Body temperature in degrees Celsius.
        alertness (str): Level of alertness, either 'alert' or 'confused or not alert'.
        inspired_oxygen (str): Type of oxygen inspired, either 'air' or 'supplemental oxygen'.
        performance_status (str): Physical activity level of the patient.

    Returns:
        float: The probability of an adverse clinical outcome within 30 days.

    Examples:
        >>> priest('male', 55, 22, 0.93, 100, 100, 37.5, 'alert', 'air', 'limited strenuous activity, can do light activity')
        0.34

       >>> priest('Male', 18, 20, 0.94, 105.0, 115.0, 35.4, 'alert', 'air', 'unrestricted normal activity')
           0.03

        >>> priest('male', 55, 22, 0.94, 95, 110, 37.2, 'alert', 'air', 'limited strenuous activity, can do light activity')
          0.34


        >>> priest('female', 85, 30, 0.88, 50, 85, 34.0, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
             0.99
    """
    # Normalize string inputs to lower case to ensure case-insensitive comparisons
    sex = sex.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()

    # Validate inputs
    if sex not in ['male', 'female']:
        raise ValueError("Invalid sex input. Must be 'male' or 'female'.")

    # Initialize the score
    priest_score = 0

    # Sex scoring
    priest_score += 1 if sex == 'male' else 0

    # Age scoring
    if age > 80:
        priest_score += 4
    elif age > 65:
        priest_score += 3
    elif age > 50:
        priest_score += 2

    # Respiratory rate scoring
    if respiratory_rate > 24:
        priest_score += 3
    elif respiratory_rate > 20:
        priest_score += 2
    elif respiratory_rate < 9:
        priest_score += 3
    elif respiratory_rate <= 11:
        priest_score += 1

    # Oxygen saturation scoring
    if oxygen_saturation < 0.92:
        priest_score += 3
    elif oxygen_saturation <= 0.93:
        priest_score += 2
    elif oxygen_saturation <= 0.95:
        priest_score += 1

    # Heart rate scoring
    if heart_rate > 130:
        priest_score += 3
    elif heart_rate > 110:
        priest_score += 2
    elif heart_rate > 90:
        priest_score += 1
    elif heart_rate < 41:
        priest_score += 3
    elif heart_rate <= 50:
        priest_score += 1

    # Systolic blood pressure scoring
    if systolic_bp > 219:
        priest_score += 3
    elif systolic_bp < 91:
        priest_score += 3
    elif systolic_bp <= 100:
        priest_score += 2
    elif systolic_bp <= 110:
        priest_score += 1

    # Temperature scoring
    if temperature > 39.0:
        priest_score += 2
    elif temperature > 38.1:
        priest_score += 1
    elif temperature < 35.1:
        priest_score += 3
    elif temperature <= 36.0:
        priest_score += 1

    # Alertness scoring
    priest_score += 3 if alertness != 'alert' else 0

    # Inspired oxygen scoring
    priest_score += 2 if inspired_oxygen == 'supplemental oxygen' else 0

    # Performance status scoring
    performance_scores = {
        'bed/chair bound, no self-care': 4,
        'limited self-care': 3,
        'limited activity, can self-care': 2,
        'limited strenuous activity, can do light activity': 1
    }
    priest_score += performance_scores.get(performance_status, 0)

    # Translate PRIEST score to probability
    if priest_score >= 26:
        return 0.99
    if priest_score >= 17:
        return 0.59
    if priest_score == 16:
        return 0.55
    if priest_score == 15:
        return 0.49
    if priest_score == 14:
        return 0.47
    if priest_score == 13:
        return 0.46
    if priest_score == 12:
        return 0.38
    if priest_score == 11:
        return 0.34
    if priest_score == 10:
        return 0.29
    if priest_score == 9:
        return 0.26
    if priest_score == 8:
        return 0.22
    if priest_score == 7:
        return 0.18
    if priest_score == 6:
        return 0.15
    if priest_score == 5:
        return 0.09
    if priest_score == 4:
        return 0.03
    if 2 <= priest_score <= 3:
        return 0.02
    if 0 <= priest_score <= 1:
        return 0.01

    return "Score"


In [58]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('male', 55, 22, 0.93, 100, 100, 37.5, 'alert', 'air', 'limited strenuous activity, can do light activity')
Expecting:
    0.34
ok
Trying:
    priest('Male', 18, 20, 0.94, 105.0, 115.0, 35.4, 'alert', 'air', 'unrestricted normal activity')
Expecting:
        0.03
**********************************************************************
File "__main__", line 31, in NoName
Failed example:
    priest('Male', 18, 20, 0.94, 105.0, 115.0, 35.4, 'alert', 'air', 'unrestricted normal activity')
Expected:
        0.03
Got:
    0.03
Trying:
    priest('male', 55, 22, 0.94, 95, 110, 37.2, 'alert', 'air', 'limited strenuous activity, can do light activity')
Expecting:
      0.34
**********************************************************************
File "__main__", line 34, in NoName
Failed example:
    priest('male', 55, 22, 0.94, 95, 110, 37.2, 'alert', 'air', 'limited strenuous activity, can do light activity')
Expected:
      0.34
Got:
    0.26
Trying:
   

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [59]:
import requests

def find_hospital(age, sex, risk):
  """
 The function find_hospital will call a REST web service to find the hospital by parameters age, sex, and risk. And returns hospital name if the call is successful otherwise None.
  It uses the definition provided by evidence from https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1

  >>> find_hospital(40, 'male', 0.18)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(22, 'female', 0.92)
        'Emory Dunwoody Medical Center'
    >>> find_hospital(30, 'female', 0.09)
        'Select Specialty Hospital - Northeast Atlanta'
  """

  url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}"

  try:
        response = requests.get(url)
        response.raise_for_status()
        response_data = response.json()
        hospital_name = response_data.get("hospital")

        return hospital_name if hospital_name else None

  except requests.RequestException as e:
        print(f"Request error: {e}")
        return None
  except ValueError as e:
        print(f"JSON decoding error: {e}")
        return None

In [60]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.18)
Expecting:
      'Southwest Hospital and Medical Center'
**********************************************************************
File "__main__", line 8, in NoName
Failed example:
    find_hospital(40, 'male', 0.18)
Expected:
      'Southwest Hospital and Medical Center'
Got:
    'Southwest Hospital and Medical Center'
Trying:
    find_hospital(22, 'female', 0.92)
Expecting:
        'Emory Dunwoody Medical Center'
**********************************************************************
File "__main__", line 10, in NoName
Failed example:
    find_hospital(22, 'female', 0.92)
Expected:
        'Emory Dunwoody Medical Center'
Got:
    'Emory Dunwoody Medical Center'
Trying:
    find_hospital(30, 'female', 0.09)
Expecting:
        'Select Specialty Hospital - Northeast Atlanta'
**********************************************************************
File "__main__", line 12, in NoName
Failed example:
    find_hospital(30, 'fema

## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [61]:
import json
import requests

def get_address(Hospital):
    """ (str) -> str
    This function retrieves the address of a hospital based on its name from the JSON file https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-, returning None if not found.


    >>> get_address("WELLSTAR ATLANTA MEDICAL CENTER")
    '303 PARKWAY DRIVE NE'

    >>> get_address("LIGHTHOUSE CARE CENTER OF AUGUSTA")
    '3100 PERIMETER PARKWAY'

    >>> get_address("APPLING HEALTHCARE SYSTEM")
    '163 EAST TOLLISON STREET'

    """

    url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = json.loads(response.text)

        if Hospital in data:
            hospital_info = data[Hospital]
            address = hospital_info.get("ADDRESS")

            if address:
                return address
        return None

    except requests.exceptions.RequestException as e:
        return None

    except Exception as e:
        return None

In [62]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("WELLSTAR ATLANTA MEDICAL CENTER")
Expecting:
    '303 PARKWAY DRIVE NE'
ok
Trying:
    get_address("LIGHTHOUSE CARE CENTER OF AUGUSTA")
Expecting:
    '3100 PERIMETER PARKWAY'
ok
Trying:
    get_address("APPLING HEALTHCARE SYSTEM")
Expecting:
    '163 EAST TOLLISON STREET'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [63]:
import requests
import csv
from io import StringIO

def process_people(file_url):
    """
    Processes patient data from the file in given location and retrieves risk, hospital and address information.
     (str) -> dict

    Args:
        file_url (str): URL to the PSV file containing patient data.

    Returns:
        dict: A dictionary mapping patient identifiers to a list of patient data including calculated risk,
              hospital name, and address.

    """
    patient_results = {}

    try:
        response = requests.get(file_url)
        response.raise_for_status()
        print("Processing patient data started")


        psv_reader = csv.reader(StringIO(response.text), delimiter='|')
        header = next(psv_reader)

        for patient_record in psv_reader:
            if len(patient_record) == 11:
                patient_id, gender, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status = patient_record
                risk_score = priest(gender, int(age), int(respiratory_rate), float(oxygen_saturation), int(heart_rate),
                                    int(systolic_bp), float(temperature), alertness, inspired_oxygen, performance_status)
                hospital_name = find_hospital(int(age), gender.lower(), risk_score)
                hospital_address = fetch_hospital_address(hospital_name)

                patient_results[patient_id] = [
                    gender, int(age), float(respiratory_rate), float(oxygen_saturation), int(heart_rate),
                    int(systolic_bp), float(temperature), alertness, inspired_oxygen, performance_status,
                    risk_score, hospital_name, hospital_address
                ]

        print("Processing patient data finished")
        return patient_results

    except requests.RequestException as request_error:
        print(f"Failed to retrieve data: {request_error}")
        return {}
    except Exception as processing_error:
        print(f"An error occurred during processing: {processing_error}")
        return {}


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [64]:
import requests
import json
def fetch_processed_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return json.loads(response.text)
    return None

processed_patient_data = fetch_processed_data("https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5")

expected_data_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
expected_patient_data = fetch_processed_data(expected_data_url)

if processed_patient_data is not None and expected_patient_data is not None:
    for patient_id, expected_values in expected_patient_data.items():
        if patient_id in processed_patient_data:
            if processed_patient_data[patient_id] == expected_values:
                print(f"Patient {patient_id}: Results match expected output.")
            else:
                print(f"Patient {patient_id}: Results do not match expected output.")
        else:
            print(f"Patient {patient_id} not found in the processed data.")

    for patient_id in processed_patient_data:
        if patient_id not in expected_patient_data:
            print(f"Patient {patient_id} in processed data but not in expected results.")

    for patient_id in expected_patient_data:
        if patient_id not in processed_patient_data:
            print(f"Patient {patient_id} in expected results but not in processed data.")
else:
    print("Failed to retrieve processed data.")


Patient E9559: Results match expected output.
Patient E9385: Results match expected output.
Patient E3067: Results match expected output.
Patient E9422: Results match expected output.
Patient E8661: Results match expected output.
Patient E6235: Results match expected output.
Patient E4451: Results match expected output.
Patient E8433: Results match expected output.
Patient E7593: Results match expected output.
Patient E3296: Results match expected output.
Patient E4157: Results match expected output.
Patient E7702: Results match expected output.
Patient E8158: Results match expected output.
Patient E4795: Results match expected output.
Patient E8800: Results match expected output.
Patient E4855: Results match expected output.
Patient E9691: Results match expected output.
Patient E4535: Results match expected output.
Patient E2360: Results match expected output.
Patient E3447: Results match expected output.
Patient E8208: Results match expected output.
Patient E4428: Results match expec

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---